In [9]:
%pip install --quiet --upgrade langchain-text-splitters langchain-community langgraph

Note: you may need to restart the kernel to use updated packages.


In [10]:
# Installing the LLM we will use 
import os
import getpass

# First we try to get from environment variable (Docker), then prompt if not available
openai_api_key = os.getenv("OPENAI_API_KEY")
if not openai_api_key:
    openai_api_key = getpass.getpass("Enter your OpenAI API key: ")
    os.environ["OPENAI_API_KEY"] = openai_api_key

from langchain.chat_models import init_chat_model

llm = init_chat_model('gpt-4o-mini', model_provider="openai")

In [11]:
# We will now install the embedding model we will use 
%pip install --quiet --upgrade   langchain-openai

Note: you may need to restart the kernel to use updated packages.


In [12]:
# We will now create an instance of that embedding model 
import os
import getpass

# Use the same environment variable as the LLM
if not os.environ.get('OPENAI_API_KEY'):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")

from langchain_openai import OpenAIEmbeddings
    
embeddingModel = OpenAIEmbeddings(model="text-embedding-3-large")

In [13]:
# the embeddings will be store in pgVector. So we install the langchain package that will allow us to do it
%pip install -qU langchain-postgres

Note: you may need to restart the kernel to use updated packages.


In [14]:
# this package help python to communicate with postgresql
%pip install --quiet psycopg2-binary

Note: you may need to restart the kernel to use updated packages.


In [15]:

import os
from langchain_postgres import PGVector

# Use environment variables for database connection
DB_HOST = os.getenv('POSTGRES_HOST', 'localhost')
DB_NAME = os.getenv('POSTGRES_DB', 'mydb')
DB_USER = os.getenv('POSTGRES_USER', 'postgres')
DB_PASSWORD = os.getenv('POSTGRES_PASSWORD', 'password')

try: 
    vector_store = PGVector(
    embeddings=embeddingModel,
    collection_name="my_docs", 
    connection=f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:5432/{DB_NAME}"
    )   
    print("✅ Vector store created successfully!")
except Exception as e:
    print(f"❌ Error: {e}")

✅ Vector store created successfully!


In [ ]:
# we'll retrieve data from the website : https://lilianweng.github.io/posts/2023-06-23-agent/
# and parse the html content to text

# web Only keep the HTML tags with class with class “post-content”, “post-title”, or “post-header”

import bs4
from langchain_community.document_loaders import WebBaseLoader

bs4_strainer = bs4.SoupStrainer(class_=("post-title", "post-header", "post-content"))

loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs={"parse_only": bs4_strainer},
)

docs = loader.load()
assert len(docs) == 1
printf(f"Total characters: {len(docs[0].page_content)}")


USER_AGENT environment variable not set, consider setting it to identify your requests.


NameError: name 'printf' is not defined